In [1]:
import pandas as pd
import numpy as np
import os

from datetime import datetime, timedelta

In [2]:
filename = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Current_BSFS_Assoc_Q2_Sample.csv'
df = pd.read_csv(filename, sep = ',')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')
df.columns = df.columns.str.replace('#', 'num')
df = df.assign(QTDsum=0,CurrentTier=0,CreateDate=0)

ts_currentData = os.path.getctime(filename)
dt_currentData = datetime.fromtimestamp(ts_currentData) - timedelta(days=31) # USED FOR TESTING-POC *** THIS SHOULD BE CHANGED
tup_currentData = dt_currentData.timetuple()

print(tup_currentData)

time.struct_time(tm_year=2019, tm_mon=3, tm_mday=16, tm_hour=15, tm_min=43, tm_sec=25, tm_wday=5, tm_yday=75, tm_isdst=-1)


In [ ]:
filename2 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/BSFS_ASSOC_Incentive.txt '
df_tierMap = pd.read_csv(filename2, sep = '\t')
df_tierMap.columns = df_tierMap.columns.str.replace(' ', '_')

ts_incentive = os.path.getctime(filename2)
dt_incentive = datetime.fromtimestamp(ts_incentive)
# print(dt_currentData)

In [ ]:
df['CreateDate'] = str(tup_currentData.tm_year) + '-' + str(tup_currentData.tm_mon) + '-' + str(tup_currentData.tm_mday)

In [ ]:
# df.columns.to_series(index=range(df.shape[1]))

In [ ]:
currentYear = datetime.now().year # use this for last transaction from G3X program
tup_now = datetime.now().timetuple() # used to create filename for CSV output

In [ ]:
calculatedData = pd.DataFrame([])
for ii in range(df.shape[0]):

    singleCusty = df.iloc[[ii]]
    singleCusty.reset_index(inplace=True)
        
    if  tup_currentData.tm_year == currentYear:

        if (tup_currentData.tm_mon == 1) | (tup_currentData.tm_mon == 2) | (tup_currentData.tm_mon == 3):
            if ((tup_currentData.tm_year % 400 == 0) | (tup_currentData.tm_year % 4 == 0)) & (tup_currentData.tm_year % 100 != 0):
                quarter = 1
                daysINqtr = 91
                runningDays = 91
                print('Leap year')
                singleCusty_select = singleCusty[
                ['Dealer_Name','Q1_TOTAL_NET','Q1_BS','Q1_FS','Q1_FZ','Q1_MB']]
                df['QTDsum'] = df.iloc[:,4]
            else:
                quarter = 1
                daysINqtr = 90
                runningDays = 90
                singleCusty_select = singleCusty[
                ['Dealer_Name','Q1_TOTAL_NET','Q1_BS','Q1_FS','Q1_FZ','Q1_MB']]
                df['QTDsum'] = df.iloc[:,4]
        elif (tup_currentData.tm_mon == 4) | (tup_currentData.tm_mon == 5) | (tup_currentData.tm_mon == 6):
            quarter = 2
            daysINqtr = 91
            runningDays = 90
            singleCusty_select = singleCusty[
            ['Dealer_Name','Q2_TOTAL_NET','Q2_BS','Q2_FS','Q2_FZ','Q2_MB']]
            df['QTDsum'] = df.iloc[:,11]
        elif (tup_currentData.tm_mon == 7) | (tup_currentData.tm_mon == 8) | (tup_currentData.tm_mon == 9):
            quarter = 3
            daysINqtr = 92
            runningDays = 90 + 91
            singleCusty_select = singleCusty[
            ['Dealer_Name','Q3_TOTAL_NET','Q3_BS','Q3_FS','Q3_FZ','Q3_MB']]
            df['QTDsum'] = df.iloc[:,18]
        else:
            quarter = 4
            daysINqtr = 92
            runningDays = 90 + 91 + 92
            singleCusty_select = singleCusty[
            ['Dealer_Name','Q4_TOTAL_NET','Q4_BS','Q4_FS','Q4_FZ','Q4_MB']]
            df['QTDsum'] = df.iloc[:,25]

        if quarter in [2,3,4]:
            numDays = tup_currentData.tm_yday - runningDays # get number of days
        else:
            numDays = tup_currentData.tm_yday # get number of days

        estimatedTires = np.floor(singleCusty_select.iloc[:,5] / (numDays/daysINqtr)).values

        if   (estimatedTires >= (df_tierMap.iloc[1,4])):
            estTier = 1
        else:
            estTier = 0
        if estTier < 1:
            tierUP = estTier + 1
        else:
            tierUP = 1

        if singleCusty.CurrentTier.values == 1:
            currentBSdollars = df_tierMap.iloc[1,1] * singleCusty_select.iloc[:,2]
            currentFSdollars = df_tierMap.iloc[1,2] * singleCusty_select.iloc[:,3]
            currentFZdollars = df_tierMap.iloc[1,3] * singleCusty_select.iloc[:,4]
        else:
            currentBSdollars = 0
            currentFSdollars = 0
            currentFZdollars = 0
        current_rebate = currentBSdollars + currentFSdollars + currentFZdollars
        
        # floor to underestimate estimated tires
        bridgeStone_est_tires = np.floor((singleCusty_select.iloc[:,2] / (numDays / daysINqtr)).values)

        bridgeStone_est_tires_rebate = bridgeStone_est_tires * df_tierMap.iloc[estTier,1]
        bridgeStone_prop = singleCusty_select.iloc[:,2] /  singleCusty_select.iloc[:,1]
        # ceil to make sure the customer surpasses the minimal 1up tire threshold
        bridgeStone_1up_tires = np.ceil((bridgeStone_prop * df_tierMap.iloc[tierUP,4]).values)
        bridgeStone_1up_tires_rebate = bridgeStone_1up_tires * df_tierMap.iloc[tierUP,1]
   
        fireStone_est_tires = np.floor((singleCusty_select.iloc[:,3] / (numDays / daysINqtr)).values)
        fireStone_est_tires_rebate = fireStone_est_tires * df_tierMap.iloc[estTier,2]
        fireStone_prop = singleCusty_select.iloc[:,3] /  singleCusty_select.iloc[:,1]
        fireStone_1up_tires = np.ceil((fireStone_prop * df_tierMap.iloc[tierUP,4]).values)
        fireStone_1up_tires_rebate = fireStone_1up_tires * df_tierMap.iloc[tierUP,2]
        
        fuzion_est_tires = np.floor((singleCusty_select.iloc[:,4] / (numDays / daysINqtr)).values)
        fuzion_est_tires_rebate = fuzion_est_tires * df_tierMap.iloc[estTier,3]
        fuzion_prop = singleCusty_select.iloc[:,4] /  singleCusty_select.iloc[:,1]
        fuzion_1up_tires = np.ceil((fuzion_prop * df_tierMap.iloc[tierUP,4]).values)
        fuzion_1up_tires_rebate = fuzion_1up_tires * df_tierMap.iloc[tierUP,3]
     
        total_est_dollars = bridgeStone_est_tires_rebate + fireStone_est_tires_rebate + fuzion_est_tires_rebate
        total_1up_dollars = bridgeStone_1up_tires_rebate + fireStone_1up_tires_rebate + fuzion_1up_tires_rebate
        total_1up_tires = bridgeStone_1up_tires + fireStone_1up_tires
        MB_est_tires = bridgeStone_est_tires + fireStone_est_tires

        current_MB_total = singleCusty_select.iloc[:,5]
        
        if estimatedTires > total_1up_tires:
            bridgeStone_1up_tires = bridgeStone_est_tires
            fireStone_1up_tires = fireStone_est_tires
            fuzion_1up_tires = fuzion_est_tires
            total_1up_tires = estimatedTires
            
            bridgeStone_1up_tires_rebate = bridgeStone_est_tires_rebate
            fireStone_1up_tires_rebate = fireStone_est_tires_rebate
            fuzion_1up_tires_rebate = fuzion_est_tires_rebate
            total_1up_dollars = total_est_dollars

        range_dollars_ll = ((total_1up_tires * df_tierMap.iloc[tierUP,2]))#.values
        range_dollars_ul = ((total_1up_tires * df_tierMap.iloc[tierUP,1]))#.values
        
###########################################################################################################################        
#         print(singleCusty)
# ###########################################################################################################################     
#         print('\nEstimated Tier:', [estTier])
        
#         print('\nEstimated Tires ~ bridgeStone:', bridgeStone_est_tires)
#         print('Estimated Tires ~ fireStone:', fireStone_est_tires)        
#         print('Estimated Tires ~ fuzion:', fuzion_est_tires)
#         print('-------------------------------------')
#         print('Estimated Tires ~ Total:',estimatedTires)
        
#         print('\nEstimated Dollars ~ bridgeStone:',bridgeStone_est_tires_rebate)
#         print('Estimated Dollars ~ fireStone:',fireStone_est_tires_rebate)
#         print('Estimated Dollars ~ fuzion:',fuzion_est_tires_rebate)
#         print('-------------------------------------')
#         print('Estimated Dollars ~ Total:',total_est_dollars)
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################
#         print('Estimated 1up Tier:', [tierUP])
        
#         print('\n1up Tires ~ bridgeStone:', bridgeStone_1up_tires)
#         print('1up Tires ~ fireStone:', fireStone_1up_tires)        
#         print('1up Tires ~ fuzion:', fuzion_1up_tires)
#         print('-------------------------------------')
#         print('Estimated 1up Tires ~ Total:',total_1up_tires)
        
#         print('\n1up Dollars ~ bridgeStone:',bridgeStone_1up_tires_rebate)
#         print('1up Dollars ~ fireStone:',fireStone_1up_tires_rebate)
#         print('1up Dollars ~ fuzion:',fuzion_1up_tires_rebate)
#         print('-------------------------------------')
#         print('1up Dollars ~ Total:',total_1up_dollars)
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################        
#         print('Range_ll:',range_dollars_ll)
#         print('Range_ul:',range_dollars_ul, '\n')
# ###########################################################################################################################
#         print(tup_currentData, '\n' 'indexNumber:', ii, '\n')
###########################################################################################################################
# ONLY MAIN BRAND (MB) TIRES QUALIFY FOR UNITS
        tempData = pd.DataFrame(
            {'estimated_tier':estTier, 'currentBSdollars':currentBSdollars, 'currentFSdollars':currentFSdollars, 'currentFZdollars':currentFZdollars, 'current_tires':current_MB_total, 'current_total_rebate':current_rebate,
             'bridgeStone_est_tires':bridgeStone_est_tires, 'fireStone_est_tires':fireStone_est_tires, 'fuzion_est_tires':fuzion_est_tires, 'current_est_tires':MB_est_tires,
             'bridgeStone_est_rebate':bridgeStone_est_tires_rebate, 'fireStone_est_rebate':fireStone_est_tires_rebate, 'fuzion_est_rebate':fuzion_est_tires_rebate, 'total_est_rebate':total_est_dollars,
             'Up_tier':tierUP,
             'bridgeStone_up_tires':bridgeStone_1up_tires, 'fireStone_up_tires':fireStone_1up_tires, 'fuzion_up_tires':fuzion_1up_tires,
             'bridgeStone_up_rebate':bridgeStone_1up_tires_rebate, 'fireStone_up_rebate':fireStone_1up_tires_rebate, 'fuzion_up_rebate':fuzion_1up_tires_rebate, 'total_up_rebate':total_1up_dollars,
             'range_lower':range_dollars_ll, 'range_upper':range_dollars_ul,'Period': quarter, 'Grain': 'Quarter'})
        calculatedData = calculatedData.append(tempData)
    else:
#         print('\nJUNK:', tup_currentData,'\n', 'indexNumber:', ii)
        range_dollars_ll = (df_tierMap.iloc[1,3] * df_tierMap.iloc[1,4])
        range_dollars_ul = (df_tierMap.iloc[1,1] * df_tierMap.iloc[1,4]) 
        tempData = pd.DataFrame(
            {'estimated_tier':0,  'currentBSdollars':0, 'currentFSdollars':0, 'currentFZdollars':0, 'current_MB_tires':0, 'current_total_rebate':0,
             'bridgeStone_est_tires':0, 'fireStone_est_tires':0, 'fuzion_est_tires':0, 'current_est_tires':0,
             'bridgeStone_est_rebate':0, 'fireStone_est_rebate':0, 'fuzion_est_rebate':0, 'current_total_est_rebate':0,
             'Up_tier':1,
             'bridgeStone_up_tires':0, 'fireStone_up_tires':0, 'fuzion_up_tires':0,
             'bridgeStone_up_rebate':0, 'fireStone_up_rebate':0, 'fuzion_up_rebate':0, 'total_up_rebate':0,
             'range_lower':[range_dollars_ll], 'range_upper':[range_dollars_ul],'period': quarter, 'grain': 'Quarter'})
        calculatedData = calculatedData.append(tempData)
###########################################################################################################################        
calculatedData = calculatedData.reset_index(drop=True)

df.loc[df['QTDsum'] < df_tierMap.iloc[1,4], 'current_tier'] = 0
df.loc[df['QTDsum'] >= df_tierMap.iloc[1,4], 'current_tier'] = 1
finalData = pd.concat([calculatedData,df], axis=1)

In [ ]:
finalData = finalData.replace([np.inf, -np.inf], np.nan).fillna(0)
# finalData.columns

In [ ]:
path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
writeFilename = path + 'CMP_BSFS_ASSOC.csv'
finalData.to_csv(writeFilename, index=False)
writeFilename_archive = path + 'csv_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
                        + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'CMP_BSFS_ASSOC_' \
                        + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
                        + '.csv'
finalData.to_csv(writeFilename_archive, index=False)

In [ ]:
os.getcwd()